In [0]:
%sql
-- LOAD BRONZE
CREATE OR REPLACE TEMPORARY VIEW node_status_bronze_view AS
SELECT 
  -- Extract node name from meta.host URL (e.g., "hl4" from "https://hl4.d.mckaylab.me:8006")
  regexp_extract(meta.host, 'https://([^.]+)', 1) as node_name,
  -- Derive status from uptime (if uptime > 0, node is online)
  CASE WHEN payload.data.uptime > 0 THEN 'online' ELSE 'offline' END as status,
  payload.data.cpu as cpu,
  payload.data.cpuinfo.cpus as maxcpu,
  payload.data.memory.used as mem,
  payload.data.memory.total as maxmem,
  payload.data.uptime as uptime,
  -- Use the collected_at_utc timestamp from meta
  CAST(meta.collected_at_utc AS TIMESTAMP) as ingestion_timestamp,
  dt as partition_date
FROM read_files('abfss://homelab-telemetry@sa01homelabdata.dfs.core.windows.net/bronze/proxmox/node_status/*/*.jsonl', format => 'json')

In [0]:
%sql
-- CREATE SILVER SCHEMA AND TABLE
CREATE SCHEMA IF NOT EXISTS main.silver;

CREATE TABLE IF NOT EXISTS main.silver.proxmox_node_status (
  node_name STRING,
  status STRING,
  cpu DOUBLE,
  maxcpu INT,
  mem BIGINT,
  maxmem BIGINT,
  uptime BIGINT,
  ingestion_timestamp TIMESTAMP,
  partition_date DATE
)
USING DELTA
PARTITIONED BY (partition_date)
LOCATION 'abfss://homelab-telemetry@sa01homelabdata.dfs.core.windows.net/silver/proxmox/node_status/';

In [0]:
%sql
-- MERGE BRONZE INTO SILVER
MERGE INTO main.silver.proxmox_node_status AS target
USING node_status_bronze_view AS source
ON target.node_name = source.node_name 
   AND target.uptime = source.uptime
WHEN NOT MATCHED THEN
  INSERT *;

In [0]:
%sql
-- CREATE GOLD SCHEMA AND TABLE
CREATE SCHEMA IF NOT EXISTS main.gold;

CREATE TABLE IF NOT EXISTS main.gold.hourly_node_performance_metrics (
  node_name STRING,
  report_date DATE,
  report_hour INT,
  avg_cpu_pct DOUBLE,
  peak_cpu_pct DOUBLE,
  min_cpu_pct DOUBLE,
  avg_mem_pct DOUBLE,
  peak_mem_pct DOUBLE,
  min_mem_pct DOUBLE,
  uptime_seconds BIGINT
)
USING DELTA
LOCATION 'abfss://homelab-telemetry@sa01homelabdata.dfs.core.windows.net/gold/proxmox/hourly_node_metrics/';

In [0]:
%sql
-- Add data to GOLD (processing ALL historical data)
INSERT INTO main.gold.hourly_node_performance_metrics
SELECT 
  node_name,
  partition_date as report_date,
  hour(ingestion_timestamp) as report_hour,
  ROUND(AVG(cpu) * 100, 2) as avg_cpu_pct,
  ROUND(MAX(cpu) * 100, 2) as peak_cpu_pct,
  ROUND(MIN(cpu) * 100, 2) as min_cpu_pct,
  ROUND(AVG(mem / maxmem) * 100, 2) as avg_mem_pct,
  ROUND(MAX(mem / maxmem) * 100, 2) as peak_mem_pct,
  ROUND(MIN(mem / maxmem) * 100, 2) as min_mem_pct,
  MAX(uptime) as uptime_seconds
FROM main.silver.proxmox_node_status
GROUP BY node_name, report_date, report_hour;

In [0]:
%sql
-- Create external location pointing to your storage container
CREATE EXTERNAL LOCATION homelab_telemetry_location
URL 'abfss://homelab-telemetry@sa01homelabdata.dfs.core.windows.net/'
WITH (STORAGE CREDENTIAL homelab_storage_credential)
COMMENT 'Homelab telemetry data storage';

In [0]:
%sql
-- This should return your metastore name if Unity Catalog is enabled
SELECT current_metastore();

In [0]:
%sql
-- Grant yourself permissions to read and write files
GRANT READ FILES, WRITE FILES ON EXTERNAL LOCATION homelab_telemetry_location TO `chris@thinklume.ca`;

In [0]:
%sql
-- Simple test to verify storage access
LIST 'abfss://homelab-telemetry@sa01homelabdata.dfs.core.windows.net/bronze/';

In [0]:
%sql
-- Check the actual structure of the JSONL files
SELECT * 
FROM read_files('abfss://homelab-telemetry@sa01homelabdata.dfs.core.windows.net/bronze/proxmox/node_status/*/*.jsonl', format => 'json')
LIMIT 3;

In [0]:
%sql
-- Verify the bronze view has data
SELECT * FROM node_status_bronze_view 
WHERE node_name = 'hl2'
ORDER BY ingestion_timestamp, node_name
LIMIT 100

In [0]:
%sql
select * from main.gold.hourly_node_performance_metrics

In [0]:
%sql
-- Create bronze, silver, gold schemas in the homelab catalog
--CREATE SCHEMA IF NOT EXISTS homelab.bronze COMMENT 'Raw ingested data';
CREATE SCHEMA IF NOT EXISTS homelab.silver COMMENT 'Cleaned and validated data';
CREATE SCHEMA IF NOT EXISTS homelab.gold COMMENT 'Business-level aggregated metrics';

In [0]:
%sql
-- Verify your new catalog exists
SHOW CATALOGS;